Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [40]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import functools
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [68]:
CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [69]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


@counter
def tsp_cost(tsp):

    if tsp[0] != tsp[-1]:
        tsp = np.append(tsp, tsp[0])
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## Cycle Crossover

In [70]:
def cycleCrossover(population, CITIES):
    crossOveredPopulation = []
    random1 = np.random.randint(0, len(CITIES)-1)  # from 0 to len(CITIES) - 1
    random2 = np.random.randint(random1 + 1, len(CITIES))  # from random1 + 1 to len(CITIES) - 1
    for i in range(10): #POPULATION_SIZE / 2
        newChild = population[i].copy()
        selectedIndexes = population[i][random1:random2+1]
        removedIndexesPopulation = []
        for k in population[i-1]:
            if k not in selectedIndexes:
                removedIndexesPopulation.append(k)
        for j in range (len(newChild)):
            if j < random1 or j > random2:
               newChild[j] = removedIndexesPopulation[0]
               removedIndexesPopulation.pop(0)
        crossOveredPopulation.append(newChild)
    return np.array(crossOveredPopulation)


## Swap Mutuation

In [72]:
def swapMutuation(population, CITIES):
    mutuated_population = []
    random1 = np.random.randint(0, len(CITIES))
    # Generate the second random number, ensuring it's different from the first
    random2 = random1
    while random2 == random1:
        random2 = np.random.randint(0, len(CITIES))

    i = 0
    for i in range(len(population)):
        random1 = np.random.randint(0, len(CITIES))
        # Generate the second random number, ensuring it's different from the first
        random2 = random1
        while random2 == random1:
            random2 = np.random.randint(0, len(CITIES))
        population[i][random1], population[i][random2] = population[i][random2], population[i][random1]
        mutuated_population.append(population[i])

    #print("-----");
    mutuated_population= np.array(mutuated_population)
    return mutuated_population

## Run Test for Vanuatu with .5 probability mutuation and crossover

In [75]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .5:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(1350.456140257617)
    tsp_cost.calls: 360036


pop 0: 1350.456140257617


(np.float64(1350.456140257617), 360036)

## Run Test for Vanuatu with .2 probability mutuation and .8 probability crossover

In [77]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .2:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(1420.3126097166087)
    tsp_cost.calls: 960060


pop 0: 1420.3126097166087


(np.float64(1420.3126097166087), 960060)

## Run Test for Italy with .5 probability mutuation and crossover

In [78]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [79]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .5:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(8439.570192617866)
    tsp_cost.calls: 1260072


pop 0: 8439.570192617866


(np.float64(8439.570192617866), 1260072)

## Run Test for Italy with .9 probability mutuation and .1 crossover

In [80]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .9:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(11178.23987495952)
    tsp_cost.calls: 1560084


pop 0: 11178.23987495952


(np.float64(11178.23987495952), 1560084)

## Run Test for US with .4 probability mutuation and .6 crossover

In [81]:
CITIES = pd.read_csv('cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abilene,32.454514,-99.738147
1,Akron,41.080456,-81.521429
2,Albuquerque,35.105552,-106.647388
3,Alexandria,38.818343,-77.082026
4,Allen,33.107224,-96.674676


In [82]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .4:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(305704.09119864355)
    tsp_cost.calls: 1860096


pop 0: 305704.09119864355


(np.float64(305704.09119864355), 1860096)

## Run test for US with  .9 probability mutuation and .1 probability crossover

In [83]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .9:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(374687.9975899627)
    tsp_cost.calls: 2160108


pop 0: 374687.9975899627


(np.float64(374687.9975899627), 2160108)

## Run test for Russia with .5 probability mutuation and crossover

In [84]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


In [86]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .5:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(129881.6707838176)
    tsp_cost.calls: 2460430


pop 0: 129881.6707838176


(np.float64(129881.6707838176), 2460430)

## Run test for Russia with .0 probability mutuation and 1 probability crossover

In [87]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .0:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(293394.3392123781)
    tsp_cost.calls: 2760442


pop 0: 293394.3392123781


(np.float64(293394.3392123781), 2760442)

## Run test for China with .0 probability mutuation and 1 probability crossover

In [88]:
CITIES = pd.read_csv('cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Acheng,45.540000,126.960000
1,Aksu,41.150000,80.250000
2,Alaer,40.515556,81.263611
3,Altay,47.840000,88.130000
4,Anbu,23.460000,116.680000


In [89]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .0:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(922865.6383007597)
    tsp_cost.calls: 3060454


pop 0: 922865.6383007597


(np.float64(922865.6383007597), 3060454)

## Run test for China with .5 probability mutuation and crossover

In [90]:
POPULATION_SIZE = 10
population = []
index = 0
MAX_GENERATION = 10000

# Create a list of indices [0, 1, 2, 3, 4]
indices = np.arange(len(CITIES))

# Generate the shuffled population
for _ in range(POPULATION_SIZE):
    shuffled_indices = np.random.permutation(indices)  # Shuffle the elements
    population.append(shuffled_indices)
all_population_cost = []

population = np.array(population)
for i in range (len(population)):
    x=tsp_cost(population[i])
      #  print(x)
    all_population_cost.append(x)

all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
min_cost_indices = sorted_indices[:3]
#print(all_population_cost[min_cost_indices])
#print(population[min_cost_indices])
#print("After mutation")

for a in range(MAX_GENERATION):
# Convert the population list to a numpy array
    population = np.array(population)
    if np.random.random() < .5:
   # print(population)
        bestPopulation = swapMutuation(population, CITIES) # mutuation with %50 possibility
    else:
        bestPopulation = cycleCrossover(population, CITIES) #crossover with %50 possibility
   # print("population.shape", population.shape)
    population = np.vstack((population, bestPopulation)).astype(int)
   # print(population.shape)
    all_population_cost = [] 
    i = 0
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]

    #print("Indices of the minimum costs:", min_cost_indices)
    #print("Minimum costs:", all_population_cost[min_cost_indices])
    population = population[min_cost_indices]

    all_population_cost = []
    for i in range (len(population)):
        x=tsp_cost(population[i])
      #  print(x)
        all_population_cost.append(x)

    all_population_cost = np.array(all_population_cost)
   # print(all_population_cost)

    sorted_indices = np.argsort(all_population_cost)

    # Select the first 10 indices (corresponding to the minimum costs)
    min_cost_indices = sorted_indices[:10]
#print(all_population_cost[min_cost_indices])
    population=population[min_cost_indices]
print("pop 0:", tsp_cost(population[0]))
ic(tsp_cost(population[0]), tsp_cost.calls)


ic| tsp_cost(population[0]): np.float64(534467.4376238532)
    tsp_cost.calls: 3360466


pop 0: 534467.4376238532


(np.float64(534467.4376238532), 3360466)